### Importo le libraries

In [1]:
import pandas as pd
import seaborn as sns

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 150)  

In [2]:
import numpy as np
import spacy 
import gensim
from nltk.tokenize import wordpunct_tokenize
import gensim.corpora as corpora

import re
import string
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
!pip install tweet-preprocessor

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


###  **Piano d'azione** del notebook:

- [Sezione 1:](#sect-1) Descriptive Statistics

- [Sezione 2:](#sect-3) Text Classification



### **Intro**: importazione del dataset, e visualizzazione dei dati

We are going to use: the "Measuring Hate Speech" Dataset by DLab at Berkeley University: https://huggingface.co/datasets/ucberkeley-dlab/measuring-hate-speech)

In [4]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 5.0 MB/s 
     |████████████████████████████████| 212 kB 53.7 MB/s 
     |████████████████████████████████| 140 kB 64.1 MB/s 
     |████████████████████████████████| 1.1 MB 41.0 MB/s 
     |████████████████████████████████| 101 kB 3.7 MB/s 
     |████████████████████████████████| 596 kB 48.7 MB/s 
     |████████████████████████████████| 127 kB 12.4 MB/s 
     |████████████████████████████████| 144 kB 47.6 MB/s 
     |████████████████████████████████| 94 kB 1.6 MB/s 
     |████████████████████████████████| 271 kB 7.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
ERROR: pip's depen

In [5]:
import datasets 
dataset = datasets.load_dataset('ucberkeley-dlab/measuring-hate-speech', 'binary')   
df = dataset['train'].to_pandas()
df.describe()

Using custom data configuration ucberkeley-dlab--measuring-hate-speech-435cebe5d62d9f6b


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/ucberkeley-dlab___parquet/ucberkeley-dlab--measuring-hate-speech-435cebe5d62d9f6b/0.0.0/7328ef7ee03eaf3f86ae40594d46a1cec86161704e02dd19f232d81eee72ade8. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

,comment_id,annotator_id,platform,sentiment,respect,insult,humiliate,status,dehumanize,violence,genocide,attack_defend,hatespeech,hate_speech_score,infitms,outfitms,annotator_severity,std_err,annotator_infitms,annotator_outfitms,hypothesis,annotator_age
count,135556.000000,135556.000000,135556.000000,135556.000000,135556.000000,135556.00000,135556.000000,135556.000000,135556.000000,135556.000000,135556.000000,135556.000000,135556.000000,135556.000000,135556.000000,135556.000000,135556.000000,135556.000000,135556.000000,135556.000000,135556.000000,135451.000000
mean,23530.416138,5567.097812,1.281352,2.954307,2.828875,2.56331,2.278638,2.698575,1.846211,1.052045,0.678413,2.625992,0.744733,-0.567428,1.034322,1.001052,-0.018817,0.300588,1.007158,1.011841,0.014589,37.910772
std,12387.194125,3230.508937,1.023542,1.231552,1.309548,1.38983,1.370876,0.898500,1.402372,1.345706,1.179598,1.114960,0.932260,2.380003,0.496867,0.791943,0.487261,0.236380,0.269876,0.675863,0.613006,11.641276
min,1.000000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-8.340000,0.100000,0.070000,-1.820000,0.020000,0.390000,0.280000,-1.578693,18.000000
25%,18148.000000,2719.000000,0.000000,2.000000,2.000000,2.00000,1.000000,2.000000,1.000000,0.000000,0.000000,2.000000,0.000000,-2.330000,0.710000,0.560000,-0.380000,0.030000,0.810000,0.670000,-0.341008,29.000000
50%,20052.000000,5602.500000,1.000000,3.000000,3.000000,3.00000,3.000000,3.000000,2.000000,0.000000,0.000000,3.000000,0.000000,-0.340000,0.960000,0.830000,-0.020000,0.340000,0.970000,0.850000,0.110405,35.000000
75%,32038.250000,8363.000000,2.000000,4.000000,4.000000,4.00000,3.000000,3.000000,3.000000,2.000000,1.000000,3.000000,2.000000,1.410000,1.300000,1.220000,0.350000,0.420000,1.170000,1.130000,0.449555,45.000000
max,50070.000000,11142.000000,3.000000,4.000000,4.000000,4.00000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,2.000000,6.300000,5.900000,9.000000,1.360000,1.900000,2.010000,9.000000,0.987511,81.000000


In [ ]:
df.head(10)

,comment_id,annotator_id,platform,sentiment,respect,insult,humiliate,status,dehumanize,violence,genocide,attack_defend,hatespeech,hate_speech_score,text,infitms,outfitms,annotator_severity,std_err,annotator_infitms,annotator_outfitms,hypothesis,target_race_asian,target_race_black,target_race_latinx,target_race_middle_eastern,target_race_native_american,target_race_pacific_islander,target_race_white,target_race_other,target_race,target_religion_atheist,target_religion_buddhist,target_religion_christian,target_religion_hindu,target_religion_jewish,target_religion_mormon,target_religion_muslim,target_religion_other,target_religion,target_origin_immigrant,target_origin_migrant_worker,target_origin_specific_country,target_origin_undocumented,target_origin_other,target_origin,target_gender_men,target_gender_non_binary,target_gender_transgender_men,target_gender_transgender_unspecified,target_gender_transgender_women,target_gender_women,target_gender_other,target_gender,target_sexuality_bisexual,target_sexuality_gay,target_sexuality_lesbian,target_sexuality_straight,target_sexuality_other,target_sexuality,target_age_children,target_age_teenagers,target_age_young_adults,target_age_middle_aged,target_age_seniors,target_age_other,target_age,target_disability_physical,target_disability_cognitive,target_disability_neurological,target_disability_visually_impaired,target_disability_hearing_impaired,target_disability_unspecific,target_disability_other,target_disability,annotator_gender,annotator_trans,annotator_educ,annotator_income,annotator_ideology,annotator_gender_men,annotator_gender_women,annotator_gender_non_binary,annotator_gender_prefer_not_to_say,annotator_gender_self_describe,annotator_transgender,annotator_cisgender,annotator_transgender_prefer_not_to_say,annotator_education_some_high_school,annotator_education_high_school_grad,annotator_education_some_college,annotator_education_college_grad_aa,annotator_education_college_grad_ba,annotator_education_professional_degree,annotator_education_masters,annotator_education_phd,annotator_income_<10k,annotator_income_10k-50k,annotator_income_50k-100k,annotator_income_100k-200k,annotator_income_>200k,annotator_ideology_extremeley_conservative,annotator_ideology_conservative,annotator_ideology_slightly_conservative,annotator_ideology_neutral,annotator_ideology_slightly_liberal,annotator_ideology_liberal,annotator_ideology_extremeley_liberal,annotator_ideology_no_opinion,annotator_race_asian,annotator_race_black,annotator_race_latinx,annotator_race_middle_eastern,annotator_race_native_american,annotator_race_pacific_islander,annotator_race_white,annotator_race_other,annotator_age,annotator_religion_atheist,annotator_religion_buddhist,annotator_religion_christian,annotator_religion_hindu,annotator_religion_jewish,annotator_religion_mormon,annotator_religion_muslim,annotator_religion_nothing,annotator_religion_other,annotator_sexuality_bisexual,annotator_sexuality_gay,annotator_sexuality_straight,annotator_sexuality_other
0,47777,10873,3,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,-3.90,"Yes indeed. She sort of reminds me of the elder lady that played the part in the movie ""Titanic"" who was telling her story!!! And I wouldn't have wanted to cover who I really am!! I would be proud...",0.81,1.88,0.36,0.34,1.35,1.23,-1.130178,True,True,True,True,True,True,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,male,no,college_grad_ba,<10k,neutral,True,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,25.0,False,False,True,False,False,False,False,False,False,False,False,True,False
1,39773,2790,2,0.0,0.0,0.0,0.0,2.0,0.0,

In [ ]:
import preprocessor as p
p.set_options(p.OPT.URL)

In [ ]:
#### Preprocessing
def clean_text(text):
  stops = set(stopwords.words("english"))    
#''', , and '''
#Make text lowercase   
  text = text.lower()

###  I remove stopwords, in addition to the other things I remove.....!!
  filtered_words = [word for word in text.split() if word not in stops]

# Removing all the tokens with lesser than 3 characters
  filtered_words = gensim.corpora.textcorpus.remove_short(filtered_words, minsize=3)
  
# Preprocessed text after stop words removal
  text = " ".join(filtered_words)    
#remove text in square brackets
##  text = re.sub(r'\[.*?\]', '', text)
# I take out URLs, using the library "tweet preprocessor"
  text = p.clean(text)
#remove punctuation   
  text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text) 
#remove words containing numbers
##  text = re.sub(r'\w*\d\w*', '', text)

#Stemming
  return gensim.parsing.preprocessing.stem_text(text)
#return text

In [ ]:
df['hatespeech'].value_counts()

0.0    80624
2.0    46021
1.0     8911
Name: hatespeech, dtype: int64

In [ ]:
def hate_speech_dumm(var):
  if var > 0:
    return 1
  else:
    return 0

In [ ]:
df['hate_speech_dumm'] = df['hatespeech'].apply(hate_speech_dumm)
df['hate_speech_dumm'].value_counts()

0    80624
1    54932
Name: hate_speech_dumm, dtype: int64

In [ ]:
df['text_clean'] = df['text'].apply(lambda x: clean_text(x))

<a name = "sect-1"></a>
### **Section 1: Descriptive Statistics**


#### **Plan**:
-Counts of words  \\
-Most Frequent words  \\
-Most Predictive Words (i.e. Lasso Regression)

In [ ]:
def number_of_words(data):
  word_list = data.split()
  return len(word_list)

#### From tutorial:  https://www.adamsmith.haus/python/answers/how-to-count-the-number-of-words-in-a-string-in-python (search: how count words python)

In [ ]:
df['number_of_words'] = df['text'].apply(number_of_words)

In [ ]:
df['number_of_words'].describe()

count    135556.000000
mean         27.012327
std          21.466210
min           1.000000
25%          11.000000
50%          20.000000
75%          37.000000
max         128.000000
Name: number_of_words, dtype: float64

In [ ]:
#### Most frequent words

In [ ]:
from collections import Counter
Counter(" ".join(df['text']).split()).most_common(100)

[('the', 106384),
 ('to', 87587),
 ('and', 78940),
 ('a', 72201),
 ('of', 61241),
 ('you', 60966),
 ('is', 46323),
 ('in', 41898),
 ('I', 40209),
 ('are', 37897),
 ('that', 37594),
 ('be', 28357),
 ('for', 26069),
 ('on', 23115),
 ('your', 21807),
 ('all', 20986),
 ('have', 20878),
 ('with', 20413),
 ('URL', 18639),
 ('people', 18056),
 ('they', 17176),
 ('this', 15555),
 ('not', 15540),
 ('their', 14981),
 ('it', 14666),
 ('like', 14351),
 ('we', 13702),
 ('them', 12716),
 ('so', 12636),
 ('just', 12384),
 ('but', 12258),
 ('my', 12236),
 ('fucking', 12131),
 ('as', 11539),
 ('will', 11421),
 ('out', 11247),
 ('should', 10865),
 ('women', 10576),
 ('our', 10572),
 ('or', 10360),
 ('up', 10000),
 ('because', 9985),
 ('fuck', 9649),
 ('if', 9647),
 ('from', 9637),
 ('You', 9587),
 ("don't", 9517),
 ('at', 9514),
 ('get', 9297),
 ('who', 9221),
 ('hate', 9145),
 ('these', 9094),
 ('her', 8913),
 ('being', 8863),
 ('do', 8738),
 ('white', 8639),
 ('go', 8577),
 ('no', 8549),
 ('by', 8516)

In [ ]:
Counter(" ".join(df['text_clean']).split()).most_common(100)

[('fuck', 29966),
 ('peopl', 23288),
 ('url', 19373),
 ('like', 16924),
 ('women', 16708),
 ('get', 15640),
 ('white', 12791),
 ('on', 12292),
 ('hate', 11787),
 ('muslim', 11771),
 ('nigger', 11582),
 ('black', 11021),
 ('bitch', 10205),
 ('kill', 10080),
 ('men', 9797),
 ('right', 9627),
 ('im', 9233),
 ('need', 8504),
 ('make', 8262),
 ('want', 8250),
 ('know', 8085),
 ('countri', 8008),
 ('back', 7954),
 ('even', 7869),
 ('us', 7424),
 ('look', 7225),
 ('human', 7216),
 ('that', 6934),
 ('would', 6717),
 ('gai', 6666),
 ('stupid', 6648),
 ('see', 6622),
 ('everi', 6494),
 ('think', 6470),
 ('love', 6077),
 ('nigga', 6039),
 ('ass', 5999),
 ('jew', 5960),
 ('illeg', 5877),
 ('world', 5724),
 ('shit', 5526),
 ('time', 5273),
 ('american', 5273),
 ('die', 5204),
 ('america', 5160),
 ('pussi', 5151),
 ('girl', 5027),
 ('christian', 4887),
 ('god', 4865),
 ('take', 4761),
 ('call', 4629),
 ('sai', 4523),
 ('wai', 4506),
 ('here', 4432),
 ('you', 4345),
 ('mani', 4341),
 ('tran', 4311),


In [ ]:
#### From tutorial/help:  https://stackoverflow.com/questions/29903025/count-most-frequent-100-words-from-sentences-in-dataframe-pandas

In [ ]:
#### Other tutorial, on the same topic: https://www.geeksforgeeks.org/find-k-frequent-words-data-set-python/

#### **Ridge Regression, for most predictive words**

#### See from the 2nd Notebook of the Series "LearningNLP Tutorials":  https://github.com/MachineLearningJournalClub/LearningNLP/blob/main/02_LearningNLP_Tutorial.ipynb 

We are going to divide between hate speech posts and non hate speech posts:

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split

In [ ]:
X = df['text_clean']
y = df['hate_speech_dumm']
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, random_state=1)

In [ ]:
vectorizer = CountVectorizer()
Xtrain_counts = vectorizer.fit_transform(Xtrain)
features_name = vectorizer.get_feature_names()

tfidf_transformer = TfidfTransformer()
Xtrain_tfidf = tfidf_transformer.fit_transform(Xtrain_counts)

Xtest_counts = vectorizer.transform(Xtest)  
Xtest_tfidf = tfidf_transformer.fit_transform(Xtest_counts)

model = LogisticRegression()
model.fit(Xtrain_tfidf, ytrain)
y_model = model.predict(Xtest_tfidf)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [ ]:
importance = model.coef_[0]

In [ ]:
df_importance = pd.DataFrame(features_name, importance)

In [ ]:
df_importance['coef'] = df_importance.index
df_importance
df_importance.rename(columns={0: "words"})

,words,coef
0.133711,00,0.133711
-0.042997,000,-0.042997
-0.022172,0000,-0.022172
0.130151,000001,0.130151
-0.143384,0001,-0.143384
...,...,...
0.123325,𝗢𝗯𝘀𝗲𝘀𝘀𝗶𝗼𝗻,0.123325
-0.062670,𝙎𝙝𝙚,-0.062670
-0.062670,𝙠𝙣𝙚𝙬,-0.062670
-0.062670,𝙬𝙖𝙨,-0.062670


In [ ]:
df_sort = df_importance.sort_values(by='coef', ascending=False)

In [ ]:
df_sort.head(50)

,0,coef
9.704065,nigger,9.704065
9.634826,faggot,9.634826
7.305292,nigga,7.305292
6.795419,fag,6.795419
4.577520,losangel,4.577520
4.047838,monkei,4.047838
3.900175,retard,3.900175
3.896596,scum,3.896596
3.862100,pig,3.862100
3.847614,fagot,3.847614


In [ ]:
df_sort.tail(50)

,0,coef
-1.375049,hispan,-1.375049
-1.377027,right,-1.377027
-1.383348,support,-1.383348
-1.389972,inspir,-1.389972
-1.390403,famili,-1.390403
-1.395192,girl,-1.395192
-1.403702,polic,-1.403702
-1.414875,dedic,-1.414875
-1.422376,howev,-1.422376
-1.422652,filipino,-1.422652


#### **Dictionary method**

In [ ]:
list_bad_words = ['bitch', 'wore']

In [ ]:
list_bad_words02 = [clean_text(i) for i in list_bad_words]
list_bad_words02

['bitch', 'wore']

In [ ]:
def hate_speech_lexic03(data):
    if any(x in data['text'] for x in list_bad_words02):
      return 1
    else:
      return 0

In [ ]:
df['text'] = df['text'].astype(str)
df['there_is_bad_word'] = df.apply(hate_speech_lexic03, axis=1)

In [ ]:
df['there_is_bad_word'].value_counts()

0    127020
1      8536
Name: there_is_bad_word, dtype: int64

In [ ]:
df[df['there_is_bad_word'] == 1]

,comment_id,annotator_id,platform,sentiment,respect,insult,humiliate,status,dehumanize,violence,genocide,attack_defend,hatespeech,hate_speech_score,text,infitms,outfitms,annotator_severity,std_err,annotator_infitms,annotator_outfitms,hypothesis,target_race_asian,target_race_black,target_race_latinx,target_race_middle_eastern,target_race_native_american,target_race_pacific_islander,target_race_white,target_race_other,target_race,target_religion_atheist,target_religion_buddhist,target_religion_christian,target_religion_hindu,target_religion_jewish,target_religion_mormon,target_religion_muslim,target_religion_other,target_religion,target_origin_immigrant,target_origin_migrant_worker,target_origin_specific_country,target_origin_undocumented,target_origin_other,target_origin,target_gender_men,target_gender_non_binary,target_gender_transgender_men,target_gender_transgender_unspecified,target_gender_transgender_women,target_gender_women,target_gender_other,target_gender,target_sexuality_bisexual,target_sexuality_gay,target_sexuality_lesbian,target_sexuality_straight,target_sexuality_other,target_sexuality,target_age_children,target_age_teenagers,target_age_young_adults,target_age_middle_aged,target_age_seniors,target_age_other,target_age,target_disability_physical,target_disability_cognitive,target_disability_neurological,target_disability_visually_impaired,target_disability_hearing_impaired,target_disability_unspecific,target_disability_other,target_disability,annotator_gender,annotator_trans,annotator_educ,annotator_income,annotator_ideology,annotator_gender_men,annotator_gender_women,annotator_gender_non_binary,annotator_gender_prefer_not_to_say,annotator_gender_self_describe,annotator_transgender,annotator_cisgender,annotator_transgender_prefer_not_to_say,annotator_education_some_high_school,annotator_education_high_school_grad,annotator_education_some_college,annotator_education_college_grad_aa,annotator_education_college_grad_ba,annotator_education_professional_degree,annotator_education_masters,annotator_education_phd,annotator_income_<10k,annotator_income_10k-50k,annotator_income_50k-100k,annotator_income_100k-200k,annotator_income_>200k,annotator_ideology_extremeley_conservative,annotator_ideology_conservative,annotator_ideology_slightly_conservative,annotator_ideology_neutral,annotator_ideology_slightly_liberal,annotator_ideology_liberal,annotator_ideology_extremeley_liberal,annotator_ideology_no_opinion,annotator_race_asian,annotator_race_black,annotator_race_latinx,annotator_race_middle_eastern,annotator_race_native_american,annotator_race_pacific_islander,annotator_race_white,annotator_race_other,annotator_age,annotator_religion_atheist,annotator_religion_buddhist,annotator_religion_christian,annotator_religion_hindu,annotator_religion_jewish,annotator_religion_mormon,annotator_religion_muslim,annotator_religion_nothing,annotator_religion_other,annotator_sexuality_bisexual,annotator_sexuality_gay,annotator_sexuality_straight,annotator_sexuality_other,hate_speech_dumm,text_clean,number_of_words,there_is_bad_word
30,39276,44,2,4.0,4.0,4.0,4.0,4.0,4.0,1.0,1.0,2.0,2.0,1.28,@KeGutta These niggas straight bitches,0.89,0.75,-0.37,0.46,1.07,0.97,-0.126042,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,female,no,college_grad_ba,50k-100k,slightly_conservative,False,True,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,53.0,False,False,True,False,False,False,False,False,False,False,False,True,False,1,kegutta nigga straight bitch,5,1
31,32811,3540,2,3.0,3.0,3.0,2.0,2.0,3.0,0.0,0.0,2.0,0.0,0.14,"Cold hoe that's a must,

### **Text Classification**

#### The idea is to try different methods, to see which one is the most accurate in terms of output metrics (i.e. accuracy, precision, recall, F1 Score)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, f1_score, precision_score, recall_score

In [ ]:
X = df['text_clean']
y = df['hate_speech_dumm']

In [ ]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, random_state=1)

In [ ]:
print(len(Xtrain))
print(len(Xtest))
print(len(ytrain))

101667
33889
101667


In [ ]:
vectorizer = CountVectorizer()
Xtrain_counts = vectorizer.fit_transform(Xtrain)

tfidf_transformer = TfidfTransformer()
Xtrain_tfidf = tfidf_transformer.fit_transform(Xtrain_counts)

In [ ]:
Xtrain_counts

<101667x36081 sparse matrix of type '<class 'numpy.int64'>'
	with 1394609 stored elements in Compressed Sparse Row format>

In [ ]:
Xtrain_tfidf

<101667x36081 sparse matrix of type '<class 'numpy.float64'>'
	with 1394609 stored elements in Compressed Sparse Row format>

In [ ]:
Xtest_counts = vectorizer.transform(Xtest)  
Xtest_tfidf = tfidf_transformer.fit_transform(Xtest_counts)

In [ ]:
model = LogisticRegression()
model.fit(Xtrain_tfidf, ytrain)
y_model = model.predict(Xtest_tfidf)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [ ]:
y_model

array([1, 0, 1, ..., 0, 0, 1])

In [ ]:
ytrain

22246     0
127168    0
30814     1
125035    0
12283     0
         ..
73349     0
109259    1
50057     0
5192      1
128037    0
Name: hate_speech_dumm, Length: 101667, dtype: int64

In [ ]:
print(accuracy_score(ytest, y_model))
print(precision_score(ytest, y_model, average='macro'))
print(recall_score(ytest, y_model, average='macro'))
print(f1_score(ytest, y_model, average='macro'))

0.8051580158753577
0.8030035841378796
0.7875894993591339
0.7929466446039417


A part that should be added is **"Cross-validation"** (I'm working toward that....)

In [ ]:
### To save the model:
import pickle
filename = '/content/gdrive/My Drive/path_to_be_added ...../model_logistic_regression.sav'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
### Same thing (i.e. model prediction) can be done with other models:
###  i.e. random forest, support vector machines, etc....

#### **Example using Random Forest Classifier**::

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model_rf = RandomForestClassifier(random_state=0)
model_rf.fit(Xtrain_tfidf, ytrain)
y_rf = model_rf.predict(Xtest_tfidf)
####!!!! Provare con XGBoost

In [ ]:
print(accuracy_score(ytest, y_rf))
print(precision_score(ytest, y_rf, average='macro'))
print(recall_score(ytest, y_rf, average='macro'))
print(f1_score(ytest, y_rf, average='macro'))

0.7914957655876538
0.7840871523893498
0.7806957040958645
0.7822227261722563


#### **Afterwards, things to do**:
- Analysis of which models works well
- Analysis of what data are not captured in your model (which are the posts that are not labelled as hate speech by your algorithm.....?? What could be some reason of this phenomenon?)
